In [92]:

import pandas as pd
import numpy as np
import json
import re
from time import time
from copy import deepcopy
from pprint import pprint
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LassoLarsCV
from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.model_selection import cross_validate, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

class ReactionDataLoader:
    def __init__(self, seed=1702913563):
        self.seed = seed
        np.random.seed(seed)
    
    def _extract_ids(self, df):
        """
        Extract unique catalyst and substrate IDs from reaction data
        
        Input DataFrame format:
        - Column 0: reaction_handle (e.g., '99_i_1_A')
        - Column 1: catalyst_id (e.g., '99_i')
        - Column 2: imine_id 
        - Column 3: thiol_id
        - Column 4: product_handle
        """
        # Extract unique catalyst IDs (e.g., '99_i')
        catalyst_ids = df.iloc[:, 1].unique()
        
        # Extract unique imine IDs and thiol IDs
        imine_ids = df.iloc[:, 2].unique()
        thiol_ids = df.iloc[:, 3].unique()
        
        print("\nExtracted IDs Summary:")
        print(f"Catalysts: {sorted(catalyst_ids)}")
        print(f"Imine IDs: {sorted(imine_ids)}")
        print(f"Thiol IDs: {sorted(thiol_ids)}")
        
        # For validation, print example mappings
        print("\nExample reaction mappings:")
        for i in range(min(5, len(df))):
            row = df.iloc[i]
            print(f"Reaction: {row.iloc[0]} -> Catalyst: {row.iloc[1]}, Imine: {row.iloc[2]}, Thiol: {row.iloc[3]}")
        
        return set(catalyst_ids), set(imine_ids), set(thiol_ids)
    
    def load_splits(self):
        """Load and process all CSV files with validation"""
        column_names = ['reaction_handle', 'catalyst_id', 'imine_id', 'thiol_id', 'product_handle']
        usecols = list(range(5))
        
        # Load datasets
        train_df = pd.read_csv('data/ER_DATA_train_set.csv', names=column_names, usecols=usecols)
        cat_test_df = pd.read_csv('data/ER_DATA_cat_test.csv', names=column_names, usecols=usecols)
        sub_test_df = pd.read_csv('data/ER_DATA_sub_test.csv', names=column_names, usecols=usecols)
        subcat_test_df = pd.read_csv('data/ER_DATA_subcat_test.csv', names=column_names, usecols=usecols)
        
        # Extract IDs from each dataset
        print("\nAnalyzing Training Set:")
        train_cats, train_imines, train_thiols = self._extract_ids(train_df)
        
        print("\nAnalyzing Catalyst Test Set:")
        test_cats, test_imines, test_thiols = self._extract_ids(cat_test_df)
        
        print("\nAnalyzing Substrate Test Set:")
        sub_cats, sub_imines, sub_thiols = self._extract_ids(sub_test_df)
        
        print("\nAnalyzing Catalyst and Substrate Test Set:")
        subcat_cats, subcat_imines, subcat_thiols = self._extract_ids(subcat_test_df)
        
        # Check for overlaps
        cat_overlap = train_cats.intersection(test_cats)
        imine_overlap = train_imines.intersection(test_imines)
        thiol_overlap = train_thiols.intersection(test_thiols)
        
        # Print validation results
        print("\nValidating splits:")
        if cat_overlap:
            print(f"Warning: Catalyst overlap between train and test: {cat_overlap}")
        if imine_overlap:
            print(f"Warning: Imine overlap between train and test: {imine_overlap}")
        if thiol_overlap:
            print(f"Warning: Thiol overlap between train and test: {thiol_overlap}")
        
        # Print dataset statistics
        print(f"\nDataset statistics:")
        print(f"Training set: {len(train_df)} reactions, {len(train_cats)} catalysts")
        print(f"Catalyst test set: {len(cat_test_df)} reactions, {len(test_cats)} catalysts")
        print(f"Substrate test set: {len(sub_test_df)} reactions")
        print(f"Subcat test set: {len(subcat_test_df)} reactions")
        
        return {
            'train_handles': train_df['reaction_handle'].tolist(),
            'unseen_cat_handles': cat_test_df['reaction_handle'].tolist(),
            'unseen_subs_handles': sub_test_df['reaction_handle'].tolist(),
            'unseen_cat_and_subs_handles': subcat_test_df['reaction_handle'].tolist(),
            'train_catalysts': train_cats,
            'train_substrates': {
                'imines': train_imines,
                'thiols': train_thiols
            },
            'test_catalysts': test_cats,
            'test_substrates': {
                'imines': test_imines,
                'thiols': test_thiols
            }
        }

def load_embeddings(file_path):
    """Load embeddings from JSON file and strip family prefix"""
    with open(file_path, 'r') as f:
        raw_embeddings = json.load(f)
    
    embeddings = {}
    family_pattern = re.compile(r'^family\d+_')
    for key, value in raw_embeddings.items():
        stripped_key = family_pattern.sub('', key)
        embeddings[stripped_key] = np.array(value)
    
    return embeddings

def multiplot_and_print(estimator, X_train, Y_train, comb_partitions, title, verbose=1, file_dpi=800):
    """Create performance plots and print metrics"""
    print(f"\n{title}")
    print(f"X_train shape: {X_train.shape}, Y_train shape: {Y_train.shape}")

    Y_train = Y_train.values.ravel()
    predicted_train = estimator.predict(X_train)
    r2_train = r2_score(Y_train, predicted_train)
    mae_train = mean_absolute_error(Y_train, predicted_train)

    if verbose:
        print(f"Train R^2: {r2_train:0.5f}, train MAE: {mae_train:0.5f}")

    fig, ax = plt.subplots(figsize=(10, 8))
    
    # Plot training data
    ax.scatter(
        Y_train,
        predicted_train,
        color="gray",
        alpha=0.5,
        label=f"Train (R^2= {r2_train:.3f}, MAE={mae_train:.3f})"
    )

    # Plot test sets
    for part_name, (X_test, Y_test, color) in comb_partitions.items():
        print(f"\nPart name: {part_name}")
        print(f"X_test shape: {X_test.shape}, Y_test shape: {Y_test.shape}")

        Y_test = Y_test.values.ravel()
        predicted_test = estimator.predict(X_test)
        r2_test = r2_score(Y_test, predicted_test)
        mae_test = mean_absolute_error(Y_test, predicted_test)

        if verbose:
            print(f"Test R^2: {r2_test:0.5f}, test MAE: {mae_test:0.5f}")

        ax.scatter(
            Y_test,
            predicted_test,
            color=color,
            alpha=0.7,
            label=f"{part_name} (R^2= {r2_test:.3f}, MAE={mae_test:.3f})"
        )
    
    ax.set_title(title)
    ax.set_xlabel("Observed $\Delta \Delta G^\u2021$ [kcal/mol]")
    ax.set_ylabel("Predicted $\Delta \Delta G^\u2021$ [kcal/mol]")
    ax.set_xlim(-3, 3)
    ax.set_ylim(-3, 3)
    ax.plot([-3, 3], [-3, 3], 'k--', alpha=0.5)

    plt.legend()
    plt.tight_layout()
    plt.savefig(f"{title}.png", dpi=file_dpi)
    plt.close()

def main():
    # Load embeddings
    embeddings = load_embeddings('/Users/utkarsh/MMLI/equicat/develop_op/final_molecule_embeddings.json')
    print(f"Loaded embeddings for {len(embeddings)} entities")
    
    # Initialize data loader
    data_loader = ReactionDataLoader()
    splits = data_loader.load_splits()
    
    # Load Y data
    Y_df = pd.read_csv('/Users/utkarsh/MMLI/equicat/src/study 2/data/Y_DATA.csv', dtype={
        'catalyst_id': str,
        'imine_id': str,
        'thiol_id': str,
        'product_id': str
    })
    print(f"Loaded Y data with {len(Y_df)} rows")
    
    # Create X data using embeddings
    X_data = []
    Y_data = []
    valid_reaction_handles = []
    missing_ids = {}

    for _, row in Y_df.iterrows():
        catalyst_id = row['catalyst_id']
        imine_id = row['imine_id']
        thiol_id = row['thiol_id']
        product_id = row['product_id']
        reaction_handle = row['reaction_handle']
        
        # Check if all required embeddings exist
        required_ids = [catalyst_id, imine_id, thiol_id, product_id]
        if all(id in embeddings for id in required_ids):
            combined_embedding = np.concatenate([
                embeddings[catalyst_id],
                embeddings[imine_id],
                embeddings[thiol_id],
                embeddings[product_id]
            ])
            
            X_data.append(combined_embedding)
            Y_data.append(row['selectivity_ddGact_kcal'])
            valid_reaction_handles.append(reaction_handle)
        else:
            missing = [id for id in required_ids if id not in embeddings]
            missing_ids[reaction_handle] = missing
    
    # Print missing IDs summary
    if missing_ids:
        print("\nMissing embeddings summary:")
        for handle, missing in missing_ids.items():
            print(f"{handle}: Missing {missing}")

    # Convert to DataFrame
    X_df = pd.DataFrame(X_data, index=valid_reaction_handles)
    Y_series = pd.Series(Y_data, index=valid_reaction_handles)

    # Create train/test splits
    valid_train_handles = [h for h in splits['train_handles'] if h in X_df.index]
    if not valid_train_handles:
        raise ValueError("No valid training examples found!")
        
    X_train = X_df.loc[valid_train_handles]
    Y_train = Y_df.loc[Y_df['reaction_handle'].isin(valid_train_handles), ["selectivity_ddGact_kcal"]]

    comb_partitions = {}

    # Create test sets
    for split_name, handles, color in [
        ("Unseen substrates", splits['unseen_subs_handles'], "green"),
        ("Unseen catalysts", splits['unseen_cat_handles'], "purple"),
        ("Unseen subs and cats", splits['unseen_cat_and_subs_handles'], "blue")
    ]:
        valid_handles = [h for h in handles if h in X_df.index]
        if valid_handles:
            X_test = X_df.loc[valid_handles]
            Y_test = Y_df.loc[Y_df['reaction_handle'].isin(valid_handles), ["selectivity_ddGact_kcal"]]
            comb_partitions[split_name] = (X_test, Y_test, color)

    # Print dataset sizes
    print("\nDataset sizes:")
    print(f"Training set: {len(X_train)} samples")
    for name, (X_test, _, _) in comb_partitions.items():
        print(f"{name} test set: {len(X_test)} samples")

    # Train models
    base_pipeline = Pipeline([('scaler', StandardScaler())])
    
    models = {
        "GBR": GradientBoostingRegressor(n_estimators=1000, ccp_alpha=1e-3),
        "SVR": SVR(kernel='poly', degree=3, epsilon=0.05),
        "RF": RandomForestRegressor(n_estimators=100, n_jobs=-1, random_state=42),
        "LL": LassoLarsCV(max_iter=5000, cv=5, n_jobs=-1),
    }

    for model_name, model in models.items():
        pipe = deepcopy(base_pipeline)
        pipe.steps.append(('model', model))
        
        print(f"\nTraining {model_name}...")
        t0 = time()
        pipe.fit(X_train, Y_train)
        print(f"{model_name}: Fitting took {time() - t0:.3f}s.")

        scores = cross_validate(
            pipe, X_train, Y_train,
            cv=5, 
            scoring=['neg_mean_absolute_error', 'r2'],
            return_train_score=True
        )
        
        print(f"\n{model_name} cross-validation scores:")
        pprint(scores)
        print(f"{model_name} Q_2: {np.mean(scores['test_r2']):.5f}")

        multiplot_and_print(pipe, X_train, Y_train, comb_partitions, f"BPA_Combinatorial_{model_name}")

    print("\nAll models trained and evaluated successfully.")

if __name__ == "__main__":
    main()


Loaded embeddings for 835 entities

Analyzing Training Set:

Extracted IDs Summary:
Catalysts: ['144_i', '157_i', '182_i', '202_i', '205_i', '207_i', '230_i', '242_i', '245_vi', '246_vi', '249_i', '251_vi', '253_i', '262_vi', '276_i', '286_vi', '328_i', '365_i', '71_vi', '72_i', '76_vi', '99_i', '99_vi']
Imine IDs: [1, 2, 3, 4]
Thiol IDs: ['A', 'B', 'C', 'D']

Example reaction mappings:
Reaction: 99_i_1_A -> Catalyst: 99_i, Imine: 1, Thiol: A
Reaction: 99_i_1_B -> Catalyst: 99_i, Imine: 1, Thiol: B
Reaction: 99_i_1_C -> Catalyst: 99_i, Imine: 1, Thiol: C
Reaction: 99_i_1_D -> Catalyst: 99_i, Imine: 1, Thiol: D
Reaction: 99_i_2_A -> Catalyst: 99_i, Imine: 2, Thiol: A

Analyzing Catalyst Test Set:

Extracted IDs Summary:
Catalysts: ['145_i', '147_i', '166_i', '1_i', '210_i', '223_i', '229_i', '229_vi', '242_vi', '245_i', '29_i', '365_vi', '371_i', '382_i', '5_i', '61_i', '73_i', '7_i', '87_i']
Imine IDs: [1, 2, 3, 4]
Thiol IDs: ['A', 'B', 'C', 'D']

Example reaction mappings:
Reaction: 1

/Users/utkarsh/miniconda3/lib/python3.11/site-packages/sklearn/ensemble/_gb.py:668: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)  # TODO: Is this still required?


GBR: Fitting took 11.912s.


/Users/utkarsh/miniconda3/lib/python3.11/site-packages/sklearn/ensemble/_gb.py:668: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)  # TODO: Is this still required?
/Users/utkarsh/miniconda3/lib/python3.11/site-packages/sklearn/ensemble/_gb.py:668: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)  # TODO: Is this still required?
/Users/utkarsh/miniconda3/lib/python3.11/site-packages/sklearn/ensemble/_gb.py:668: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)  # TODO: Is this still required?
/Users/utkarsh/miniconda3/lib/python3.11/site-packages/sklearn/ensemble/_gb.py:668


GBR cross-validation scores:
{'fit_time': array([9.14117098, 9.0605092 , 9.02740598, 8.89590001, 9.39798927]),
 'score_time': array([0.00789285, 0.00304174, 0.00302219, 0.00298095, 0.00310612]),
 'test_neg_mean_absolute_error': array([-0.30482696, -0.47648154, -0.61239168, -0.34053306, -0.2450179 ]),
 'test_r2': array([ 0.56327723, -0.68019026,  0.23501418,  0.34683033,  0.51884659]),
 'train_neg_mean_absolute_error': array([-0.1226972 , -0.12595403, -0.12272223, -0.11838078, -0.12793399]),
 'train_r2': array([0.94387383, 0.93969608, 0.92878259, 0.94428549, 0.94511225])}
GBR Q_2: 0.19676

BPA_Combinatorial_GBR
X_train shape: (368, 768), Y_train shape: (368, 1)
Train R^2: 0.93589, train MAE: 0.12777

Part name: Unseen substrates
X_test shape: (207, 768), Y_test shape: (207, 1)
Test R^2: 0.82993, test MAE: 0.22361

Part name: Unseen catalysts
X_test shape: (304, 768), Y_test shape: (304, 1)
Test R^2: -0.22306, test MAE: 0.58371

Part name: Unseen subs and cats
X_test shape: (171, 768), 

...


RF cross-validation scores:
{'fit_time': array([0.48172808, 0.50019002, 0.41956401, 0.4184761 , 0.43400073]),
 'score_time': array([0.01563001, 0.01669097, 0.01578379, 0.01486492, 0.0172081 ]),
 'test_neg_mean_absolute_error': array([-0.47399529, -0.51231977, -0.63220229, -0.43417511, -0.13133392]),
 'test_r2': array([-0.027926  , -1.01950141,  0.27345883,  0.06087114,  0.82465065]),
 'train_neg_mean_absolute_error': array([-0.05463329, -0.05556086, -0.05589263, -0.04460317, -0.05395946]),
 'train_r2': array([0.98868555, 0.98827987, 0.98255489, 0.99144028, 0.99024977])}
RF Q_2: 0.02231

BPA_Combinatorial_RF
X_train shape: (368, 768), Y_train shape: (368, 1)
Train R^2: 0.98977, train MAE: 0.05130

Part name: Unseen substrates
X_test shape: (207, 768), Y_test shape: (207, 1)
Test R^2: 0.83778, test MAE: 0.20929

Part name: Unseen catalysts
X_test shape: (304, 768), Y_test shape: (304, 1)
Test R^2: -0.15801, test MAE: 0.59535

Part name: Unseen subs and cats
X_test shape: (171, 768), Y_t

/Users/utkarsh/miniconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:1310: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LL: Fitting took 2.003s.


/Users/utkarsh/miniconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:1310: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/utkarsh/miniconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:1310: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/utkarsh/miniconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:1310: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/utkarsh/miniconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:1310: DataConversionWarning: A column-vector y was passed when a 1d array was expec


LL cross-validation scores:
{'fit_time': array([0.592731  , 0.54881787, 0.21911001, 0.21605873, 0.21043277]),
 'score_time': array([0.00548291, 0.00436497, 0.00290704, 0.00291991, 0.00811791]),
 'test_neg_mean_absolute_error': array([-0.60886078, -0.69244602, -0.71603673, -0.52813878, -0.34949318]),
 'test_r2': array([-0.90525404, -2.06734921,  0.02966678, -0.43018774, -0.24100981]),
 'train_neg_mean_absolute_error': array([-0.18621511, -0.26174785, -0.18000569, -0.24388944, -0.18683855]),
 'train_r2': array([0.87147958, 0.71966497, 0.83315972, 0.75848981, 0.88377218])}
LL Q_2: -0.72283

BPA_Combinatorial_LL
X_train shape: (368, 768), Y_train shape: (368, 1)
Train R^2: 0.25073, train MAE: 0.45696

Part name: Unseen substrates
X_test shape: (207, 768), Y_test shape: (207, 1)
Test R^2: 0.23083, test MAE: 0.46773

Part name: Unseen catalysts
X_test shape: (304, 768), Y_test shape: (304, 1)
Test R^2: 0.03397, test MAE: 0.52779

Part name: Unseen subs and cats
X_test shape: (171, 768), Y_t

The below code is the one where only 5 and E are considered in the test set. 

In [1]:

import pandas as pd
import numpy as np
import json
import re
from time import time
from copy import deepcopy
from pprint import pprint
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LassoLarsCV
from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.model_selection import cross_validate, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

class ReactionDataLoader:
    def __init__(self, seed=1702913563):
        self.seed = seed
        np.random.seed(seed)
    
    def _extract_ids(self, df):
        """
        Extract unique catalyst and substrate IDs from reaction data
        
        Input DataFrame format:
        - Column 0: reaction_handle (e.g., '99_i_1_A')
        - Column 1: catalyst_id (e.g., '99_i')
        - Column 2: imine_id 
        - Column 3: thiol_id
        - Column 4: product_handle
        """
        catalyst_ids = df['catalyst_id'].unique()
        imine_ids = df['imine_id'].unique()
        thiol_ids = df['thiol_id'].unique()
        
        print("\nExtracted IDs Summary:")
        print(f"Catalysts: {sorted(catalyst_ids)}")
        print(f"Imine IDs: {sorted(imine_ids)}")
        print(f"Thiol IDs: {sorted(thiol_ids)}")
        
        print("\nExample reaction mappings:")
        for i in range(min(5, len(df))):
            row = df.iloc[i]
            print(f"Reaction: {row['reaction_handle']} -> "
                  f"Catalyst: {row['catalyst_id']}, "
                  f"Imine: {row['imine_id']}, "
                  f"Thiol: {row['thiol_id']}")
        
        return set(catalyst_ids), set(imine_ids), set(thiol_ids)

    def _create_strict_splits(self, df):
        """
        Create splits ensuring proper separation of catalysts and substrates.
        
        Training:
            - Catalysts: Training set catalysts
            - Substrates: Imines [1,2,3,4], Thiols [A,B,C,D]
        
        Test:
            - New Catalysts: Test set catalysts
            - New Substrates: ONLY Imine [5], ONLY Thiol [E]
        """
        # Define train/test splits
        train_imines = [1, 2, 3, 4]
        train_thiols = ['A', 'B', 'C', 'D']
        test_imines = [5]
        test_thiols = ['E']

        # Split catalysts (using deterministic split from dataset)
        train_catalysts = [
            '144_i', '157_i', '182_i', '202_i', '205_i', '207_i', '230_i', 
            '242_i', '245_vi', '246_vi', '249_i', '251_vi', '253_i', '262_vi', 
            '276_i', '286_vi', '328_i', '365_i', '71_vi', '72_i', '76_vi', 
            '99_i', '99_vi'
        ]
        test_catalysts = [cat for cat in df['catalyst_id'].unique() if cat not in train_catalysts]

        # Create masks for each quadrant
        # Training Set: Training catalysts + Training substrates only
        train_mask = (
            df['catalyst_id'].isin(train_catalysts) & 
            df['imine_id'].isin(train_imines) & 
            df['thiol_id'].isin(train_thiols)
        )
        
        # Catalyst Test Set: Test catalysts + Training substrates only
        cat_test_mask = (
            df['catalyst_id'].isin(test_catalysts) & 
            df['imine_id'].isin(train_imines) & 
            df['thiol_id'].isin(train_thiols)
        )
        
        # Substrate Test Set: Training catalysts + ONLY test substrates
        sub_test_mask = (
            df['catalyst_id'].isin(train_catalysts) & 
            df['imine_id'].isin(test_imines) &  # Only imine 5
            df['thiol_id'].isin(test_thiols)    # Only thiol E
        )
        
        # Combined Test Set: Test catalysts + ONLY test substrates
        subcat_test_mask = (
            df['catalyst_id'].isin(test_catalysts) & 
            df['imine_id'].isin(test_imines) &  # Only imine 5
            df['thiol_id'].isin(test_thiols)    # Only thiol E
        )

        splits = {
            'train': df[train_mask].copy(),
            'cat_test': df[cat_test_mask].copy(),
            'sub_test': df[sub_test_mask].copy(),
            'subcat_test': df[subcat_test_mask].copy()
        }

        # Verify splits
        print("\nDetailed Split Verification:")
        for name, split_df in splits.items():
            print(f"\n{name} split:")
            cats = sorted(split_df['catalyst_id'].unique())
            imines = sorted(split_df['imine_id'].unique())
            thiols = sorted(split_df['thiol_id'].unique())
            
            print(f"Number of reactions: {len(split_df)}")
            print(f"Catalysts: {cats}")
            print(f"Imines: {imines}")
            print(f"Thiols: {thiols}")
            
            if name in ['sub_test', 'subcat_test']:
                if set(imines) != set(test_imines) or set(thiols) != set(test_thiols):
                    raise ValueError(f"{name} contains non-test substrates!")
                    
                reactions = split_df.apply(
                    lambda x: f"{x['catalyst_id']}_{x['imine_id']}_{x['thiol_id']}", 
                    axis=1
                ).tolist()
                print("\nExample reactions in this split:")
                for r in reactions[:25]:
                    print(f"  {r}")

        return splits

    def load_splits(self):
        """Load and process all data with proper validation"""
        # Load full dataset
        df = pd.read_csv('data/Y_DATA.csv')
        
        # Create strict splits
        splits = self._create_strict_splits(df)
        
        # Extract and validate IDs for each split
        print("\nAnalyzing Training Set:")
        train_cats, train_imines, train_thiols = self._extract_ids(splits['train'])
        
        print("\nAnalyzing Catalyst Test Set:")
        test_cats, test_imines, test_thiols = self._extract_ids(splits['cat_test'])
        
        print("\nAnalyzing Substrate Test Set:")
        sub_cats, sub_imines, sub_thiols = self._extract_ids(splits['sub_test'])
        
        print("\nAnalyzing Catalyst and Substrate Test Set:")
        subcat_cats, subcat_imines, subcat_thiols = self._extract_ids(splits['subcat_test'])
        
        return {
            'train_handles': splits['train']['reaction_handle'].tolist(),
            'unseen_cat_handles': splits['cat_test']['reaction_handle'].tolist(),
            'unseen_subs_handles': splits['sub_test']['reaction_handle'].tolist(),
            'unseen_cat_and_subs_handles': splits['subcat_test']['reaction_handle'].tolist(),
            'train_catalysts': train_cats,
            'train_substrates': {
                'imines': train_imines,
                'thiols': train_thiols
            },
            'test_catalysts': test_cats,
            'test_substrates': {
                'imines': sub_imines,
                'thiols': sub_thiols
            }
        }

def load_embeddings(file_path):
    """Load embeddings from JSON file and strip family prefix"""
    with open(file_path, 'r') as f:
        raw_embeddings = json.load(f)
    
    embeddings = {}
    family_pattern = re.compile(r'^family\d+_')
    for key, value in raw_embeddings.items():
        stripped_key = family_pattern.sub('', key)
        embeddings[stripped_key] = np.array(value)
    
    return embeddings

def multiplot_and_print(estimator, X_train, Y_train, comb_partitions, title, verbose=1, file_dpi=800):
    """Create performance plots and print metrics"""
    print(f"\n{title}")
    print(f"X_train shape: {X_train.shape}, Y_train shape: {Y_train.shape}")

    Y_train = Y_train.values.ravel()
    predicted_train = estimator.predict(X_train)
    r2_train = r2_score(Y_train, predicted_train)
    mae_train = mean_absolute_error(Y_train, predicted_train)

    if verbose:
        print(f"Train R^2: {r2_train:0.5f}, train MAE: {mae_train:0.5f}")

    fig, ax = plt.subplots(figsize=(10, 8))
    
    # Plot training data
    ax.scatter(
        Y_train,
        predicted_train,
        color="gray",
        alpha=0.5,
        label=f"Train (R^2= {r2_train:.3f}, MAE={mae_train:.3f})"
    )

    # Plot test sets
    for part_name, (X_test, Y_test, color) in comb_partitions.items():
        print(f"\nPart name: {part_name}")
        print(f"X_test shape: {X_test.shape}, Y_test shape: {Y_test.shape}")

        Y_test = Y_test.values.ravel()
        predicted_test = estimator.predict(X_test)
        r2_test = r2_score(Y_test, predicted_test)
        mae_test = mean_absolute_error(Y_test, predicted_test)

        if verbose:
            print(f"Test R^2: {r2_test:0.5f}, test MAE: {mae_test:0.5f}")

        ax.scatter(
            Y_test,
            predicted_test,
            color=color,
            alpha=0.7,
            label=f"{part_name} (R^2= {r2_test:.3f}, MAE={mae_test:.3f})"
        )
    
    ax.set_title(title)
    ax.set_xlabel("Observed $\Delta \Delta G^\u2021$ [kcal/mol]")
    ax.set_ylabel("Predicted $\Delta \Delta G^\u2021$ [kcal/mol]")
    ax.set_xlim(-3, 3)
    ax.set_ylim(-3, 3)
    ax.plot([-3, 3], [-3, 3], 'k--', alpha=0.5)

    plt.legend()
    plt.tight_layout()
    plt.savefig(f"{title}.png", dpi=file_dpi)
    plt.close()

def main():
    # Load embeddings
    embeddings = load_embeddings('/Users/utkarsh/MMLI/equicat/develop_op/final_molecule_embeddings.json')
    print(f"Loaded embeddings for {len(embeddings)} entities")
    
    # Initialize data loader
    data_loader = ReactionDataLoader()
    splits = data_loader.load_splits()
    
    # Load Y data
    Y_df = pd.read_csv('/Users/utkarsh/MMLI/equicat/src/study 2/data/Y_DATA.csv', dtype={
        'catalyst_id': str,
        'imine_id': str,
        'thiol_id': str,
        'product_id': str
    })
    print(f"Loaded Y data with {len(Y_df)} rows")
    
    # Create X data using embeddings
    X_data = []
    Y_data = []
    valid_reaction_handles = []
    missing_ids = {}

    for _, row in Y_df.iterrows():
        catalyst_id = row['catalyst_id']
        imine_id = row['imine_id']
        thiol_id = row['thiol_id']
        product_id = row['product_id']
        reaction_handle = row['reaction_handle']
        
        # Check if all required embeddings exist
        required_ids = [catalyst_id, imine_id, thiol_id, product_id]
        if all(id in embeddings for id in required_ids):
            combined_embedding = np.concatenate([
                embeddings[catalyst_id],
                embeddings[imine_id],
                embeddings[thiol_id],
                embeddings[product_id]
            ])
            
            X_data.append(combined_embedding)
            Y_data.append(row['selectivity_ddGact_kcal'])
            valid_reaction_handles.append(reaction_handle)
        else:
            missing = [id for id in required_ids if id not in embeddings]
            missing_ids[reaction_handle] = missing
    
    # Print missing IDs summary
    if missing_ids:
        print("\nMissing embeddings summary:")
        for handle, missing in missing_ids.items():
            print(f"{handle}: Missing {missing}")

    # Convert to DataFrame
    X_df = pd.DataFrame(X_data, index=valid_reaction_handles)
    Y_series = pd.Series(Y_data, index=valid_reaction_handles)

    # Create train/test splits
    valid_train_handles = [h for h in splits['train_handles'] if h in X_df.index]
    if not valid_train_handles:
        raise ValueError("No valid training examples found!")
        
    X_train = X_df.loc[valid_train_handles]
    Y_train = Y_df.loc[Y_df['reaction_handle'].isin(valid_train_handles), ["selectivity_ddGact_kcal"]]

    comb_partitions = {}

    # Create test sets
    for split_name, handles, color in [
        ("Unseen substrates", splits['unseen_subs_handles'], "green"),
        ("Unseen catalysts", splits['unseen_cat_handles'], "purple"),
        ("Unseen subs and cats", splits['unseen_cat_and_subs_handles'], "blue")
    ]:
        valid_handles = [h for h in handles if h in X_df.index]
        if valid_handles:
            X_test = X_df.loc[valid_handles]
            Y_test = Y_df.loc[Y_df['reaction_handle'].isin(valid_handles), ["selectivity_ddGact_kcal"]]
            comb_partitions[split_name] = (X_test, Y_test, color)

    # Print dataset sizes
    print("\nDataset sizes:")
    print(f"Training set: {len(X_train)} samples")
    for name, (X_test, _, _) in comb_partitions.items():
        print(f"{name} test set: {len(X_test)} samples")

    # Train models
    base_pipeline = Pipeline([('scaler', StandardScaler())])
    
    models = {
        "GBR": GradientBoostingRegressor(n_estimators=500, ccp_alpha=1e-3),
        "SVR": SVR(kernel='poly', degree=3, epsilon=0.05),
        "RF": RandomForestRegressor(n_estimators=500, n_jobs=-1, random_state=7),
        "LL": LassoLarsCV(max_iter=5000, cv=5, n_jobs=-1),
    }

    for model_name, model in models.items():
        pipe = deepcopy(base_pipeline)
        pipe.steps.append(('model', model))
        
        print(f"\nTraining {model_name}...")
        t0 = time()
        pipe.fit(X_train, Y_train)
        print(f"{model_name}: Fitting took {time() - t0:.3f}s.")

        scores = cross_validate(
            pipe, X_train, Y_train,
            cv=5, 
            scoring=['neg_mean_absolute_error', 'r2'],
            return_train_score=True
        )
        
        print(f"\n{model_name} cross-validation scores:")
        pprint(scores)
        print(f"{model_name} Q_2: {np.mean(scores['test_r2']):.5f}")

        multiplot_and_print(pipe, X_train, Y_train, comb_partitions, f"BPA_Combinatorial_{model_name}")

    print("\nAll models trained and evaluated successfully.")

if __name__ == "__main__":
    main()

Loaded embeddings for 835 entities

Detailed Split Verification:

train split:
Number of reactions: 368
Catalysts: ['144_i', '157_i', '182_i', '202_i', '205_i', '207_i', '230_i', '242_i', '245_vi', '246_vi', '249_i', '251_vi', '253_i', '262_vi', '276_i', '286_vi', '328_i', '365_i', '71_vi', '72_i', '76_vi', '99_i', '99_vi']
Imines: [1, 2, 3, 4]
Thiols: ['A', 'B', 'C', 'D']

cat_test split:
Number of reactions: 304
Catalysts: ['145_i', '147_i', '166_i', '1_i', '210_i', '223_i', '229_i', '229_vi', '242_vi', '245_i', '29_i', '365_vi', '371_i', '382_i', '5_i', '61_i', '73_i', '7_i', '87_i']
Imines: [1, 2, 3, 4]
Thiols: ['A', 'B', 'C', 'D']

sub_test split:
Number of reactions: 23
Catalysts: ['144_i', '157_i', '182_i', '202_i', '205_i', '207_i', '230_i', '242_i', '245_vi', '246_vi', '249_i', '251_vi', '253_i', '262_vi', '276_i', '286_vi', '328_i', '365_i', '71_vi', '72_i', '76_vi', '99_i', '99_vi']
Imines: [5]
Thiols: ['E']

Example reactions in this split:
  144_i_5_E
  157_i_5_E
  182_i_5

/Users/utkarsh/miniconda3/lib/python3.11/site-packages/sklearn/ensemble/_gb.py:668: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)  # TODO: Is this still required?


GBR: Fitting took 5.904s.


/Users/utkarsh/miniconda3/lib/python3.11/site-packages/sklearn/ensemble/_gb.py:668: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)  # TODO: Is this still required?
/Users/utkarsh/miniconda3/lib/python3.11/site-packages/sklearn/ensemble/_gb.py:668: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)  # TODO: Is this still required?
/Users/utkarsh/miniconda3/lib/python3.11/site-packages/sklearn/ensemble/_gb.py:668: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)  # TODO: Is this still required?
/Users/utkarsh/miniconda3/lib/python3.11/site-packages/sklearn/ensemble/_gb.py:668


GBR cross-validation scores:
{'fit_time': array([4.9062891 , 4.53885913, 4.65147185, 4.65691018, 4.53855324]),
 'score_time': array([0.00512505, 0.00329208, 0.00311708, 0.00320578, 0.00337791]),
 'test_neg_mean_absolute_error': array([-0.41069516, -0.42526388, -0.62571789, -0.62317576, -0.24722637]),
 'test_r2': array([ 0.16699273, -0.35320623,  0.06231398, -0.72928097,  0.48796157]),
 'train_neg_mean_absolute_error': array([-0.12402647, -0.12377748, -0.12210599, -0.12118226, -0.12319598]),
 'train_r2': array([0.94292632, 0.94140733, 0.92870833, 0.94172006, 0.94888819])}
GBR Q_2: -0.07304

BPA_Combinatorial_GBR
X_train shape: (368, 768), Y_train shape: (368, 1)
Train R^2: 0.93751, train MAE: 0.12750

Part name: Unseen substrates
X_test shape: (23, 768), Y_test shape: (23, 1)
Test R^2: 0.82377, test MAE: 0.24455

Part name: Unseen catalysts
X_test shape: (304, 768), Y_test shape: (304, 1)
Test R^2: -0.26485, test MAE: 0.61214

Part name: Unseen subs and cats
X_test shape: (19, 768), Y_

...


RF cross-validation scores:
{'fit_time': array([2.64140892, 2.24109411, 1.96061301, 2.10982776, 2.02295899]),
 'score_time': array([0.05497217, 0.02999091, 0.03041792, 0.02985811, 0.02908301]),
 'test_neg_mean_absolute_error': array([-0.37797197, -0.48079649, -0.5826621 , -0.56151953, -0.23957457]),
 'test_r2': array([ 0.18333158, -0.82271057,  0.24526537, -0.59721609,  0.54954889]),
 'train_neg_mean_absolute_error': array([-0.05273145, -0.05232615, -0.05087525, -0.04450787, -0.05122391]),
 'train_r2': array([0.98961871, 0.98905567, 0.98580256, 0.99122029, 0.99050928])}
RF Q_2: -0.08836

BPA_Combinatorial_RF
X_train shape: (368, 768), Y_train shape: (368, 1)
Train R^2: 0.99009, train MAE: 0.04901

Part name: Unseen substrates
X_test shape: (23, 768), Y_test shape: (23, 1)
Test R^2: 0.85847, test MAE: 0.20301

Part name: Unseen catalysts
X_test shape: (304, 768), Y_test shape: (304, 1)
Test R^2: -0.11372, test MAE: 0.56723

Part name: Unseen subs and cats
X_test shape: (19, 768), Y_tes

/Users/utkarsh/miniconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:1310: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LL: Fitting took 2.327s.


/Users/utkarsh/miniconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:1310: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/utkarsh/miniconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:1310: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/utkarsh/miniconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:1310: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/utkarsh/miniconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:1310: DataConversionWarning: A column-vector y was passed when a 1d array was expec


LL cross-validation scores:
{'fit_time': array([0.56623483, 0.55230308, 0.24621606, 0.20464611, 0.20733809]),
 'score_time': array([0.00599313, 0.00441504, 0.00442815, 0.00563788, 0.00291014]),
 'test_neg_mean_absolute_error': array([-0.41686895, -0.44020302, -0.62516978, -0.59061301, -0.2379075 ]),
 'test_r2': array([ 0.2372896 , -0.30975998,  0.00749119, -0.79954138,  0.52490814]),
 'train_neg_mean_absolute_error': array([-0.27232023, -0.1831481 , -0.15294249, -0.16361138, -0.1863519 ]),
 'train_r2': array([0.70431977, 0.85361564, 0.88177425, 0.88250215, 0.88369876])}
LL Q_2: -0.06792

BPA_Combinatorial_LL
X_train shape: (368, 768), Y_train shape: (368, 1)
Train R^2: 0.86754, train MAE: 0.17734

Part name: Unseen substrates
X_test shape: (23, 768), Y_test shape: (23, 1)
Test R^2: -1.75760, test MAE: 1.12638

Part name: Unseen catalysts
X_test shape: (304, 768), Y_test shape: (304, 1)
Test R^2: -0.24233, test MAE: 0.55666

Part name: Unseen subs and cats
X_test shape: (19, 768), Y_te